In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [ ]:
import pandas as pd, numpy as np 
from travis_attack.config import Config
from travis_attack.insights import get_training_dfs, _prepare_df_concat
from travis_attack.utils import display_all
from IPython.core.debugger import set_trace
import logging 
logger = logging.getLogger("run")

In [ ]:
cfg = Config()
run_id = 'cdy5d5yu'
run_name = "hopeful-snowflake-297"
path_run = f"{cfg.path_checkpoints}{run_name}/"

## Looking at some examples 

Below we just look at some examples to get a feel for what is going on. 

In [ ]:
def get_interesting_idx(df, n):
    def get_idx_with_top_column_values(cname, n=5, ascending=False):
        return df[['idx',cname]].\
            drop_duplicates().\
            sort_values(cname, ascending=ascending)\
            ['idx'][0:n].values.tolist()
    
    def sample_idx_with_label_flips(n=5): 
        df1 = df[['idx','label_flip']].query("label_flip!=0")
        if len(df1) == 0 : print("No label flips detected"); return None
        else: return df1.drop_duplicates()['idx'].sample(n).values.tolist()
    
    idx_d = dict(
        random = df.idx.drop_duplicates().sample(n).tolist(),
        label_flips = sample_idx_with_label_flips(n=n),
        idx_n_unique_pp  = get_idx_with_top_column_values('idx_n_unique_pp',n=n,ascending=False),
       # idx_n_pp_changes = get_idx_with_top_column_values('idx_n_pp_changes',n=n,ascending=False),
        low_sts = get_idx_with_top_column_values('sts_score',n=n,ascending=True)
    )
    return idx_d

def print_stats(df, idx_d, key,i):
    print("\n###############\n")
    print(key, i, "\n")
    if idx_d[key] is None: return
    idx = idx_d[key][i]
    # Setup 
    df1 = df.query('idx==@idx')
    orig = pd.unique(df1['orig_l'])[0]
    print("Original:", orig)
    print("Original label", pd.unique(df1['orig_label'])[0] )
    pp_all = list(df1['pp_l'])
    #print("All paraphrases", pp_all)
    pp_unique = list(pd.unique(df1['pp_l']))
    n_pp_unique = len(pp_unique)

    # showing a "timeline" of how the paraphrases change over the epochs
    g_fields = ["pp_l","pp_truelabel_probs","vm_score","sts_score","reward","label_flip"]
    #g_fields = ["pp_l","vm_score"]
    g = df1.groupby(g_fields).agg({'epoch' : lambda x: list(x)})
    g = g.sort_values(by='epoch', key = lambda col: col.map(lambda x: np.min(x)))
    print("Unique paraphrases:", n_pp_unique)
    print("How the paraphrases change:")
    display_all(g)

    # Showing a dataframe of the few best paraphrases
    best_pps = df1.sort_values('pp_truelabel_probs').iloc[0]
    print("Best Paraphrase")
    display_all(best_pps.to_frame().T)
        
def print_interesting_text_stats(df, n): 
    idx_d = get_interesting_idx(df, n)
    for key in idx_d.keys():
        for i in range(n): 
            print_stats(df, idx_d, key,i)

In [ ]:
split = 'training_step'
df_d = get_training_dfs(path_run, postprocessed=True)
idx_d = get_interesting_idx(df_d[split], n=2)
print_interesting_text_stats(df_d[split], n=2)


###############

random 0 

Original: I like this movie
Original label 1
Unique paraphrases: 29
How the paraphrases change:


,,,,,,epoch
pp_l,pp_truelabel_probs,vm_score,sts_score,reward,label_flip,
I like this movie.,0.82852,-0.12236,0.99158,0.37867,0,"[1, 2, 3, 6, 8, 9, 11, 12, 20, 21, 22, 23, 24, 25, 27, 30, 31, 34, 36, 39, 40, 41, 44, 46, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 66, 68, 69, 70, 71, 72, 73, 76, 77, 78, 80, 81, 83, 85, 87, 88, 89, 90, 92, 93, 94, 96, 97, 98, 99, 100, 102, 103, 104, 105, 106, 107, 108, 109, 111, 112, 113, 114, 115, 116, 117, 119, 120, 121, 122, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 136, 137, 138, 139, ...]"
I like it.,0.80296,-0.09679,0.70295,0.00000,0,"[4, 13, 176, 204]"
I like the movie.,0.77841,-0.07224,0.91621,0.43381,0,"[5, 7, 15, 19, 28, 35, 38, 45, 47, 67, 74, 82, 84, 95, 101, 110, 123, 124, 141, 182, 194, 199, 239, 256, 293]"
This is a movie I like.,0.81837,-0.11220,0.87988,0.40128,0,[10]
....,0.48935,0.21682,0.15566,0.00000,1,"[14, 32, 75, 161, 288]"
I like movies.,0.60663,0.09954,0.65285,0.00000,0,"[16, 33, 253, 267]"
This movie is good.,0.92876,-0.22259,0.83795,0.31348,0,"[17, 146, 188]"
..empsempsempsempsempsempsempsempsempsempsempsempsempsempsempsemps,0.34323,0.36294,0.01656,0.00000,1,[18]
I like this.,0.78075,-0.07458,0.66832,0.00000,0,"[26, 198]"


Best Paraphrase


,idx,epoch,orig_l,pp_l,orig_truelabel_probs,pp_truelabel_probs,pp_predclass_probs,orig_label,pp_predclass,label_flip,vm_score,sts_score,reward,pp_logp,loss,batch_num,global_step,acc_num,loss_sum,loss_batch,label_flip_fraction,orig_length,orig_batch_size,pp_length,pp_batch_size,time_generate_pp,time_loss_fn,time_reward_fn,time_vm_scores,time_sts_scores,time_pp_logp,time_log_entropy,time_log_token_probabilities,time_loss_fn_loss_calc,time_backwards,time_calc_gradient_norm,time_opt_step_and_calc_param_norm,time_opt_step,idx_n_unique_pp,idx_n_pp_changes,epoch_of_first_label_flip,n_words_orig,n_sentences_orig,n_punctuation_orig,n_digits_orig,n_letters_orig,n_words_pp,n_sentences_pp,n_punctuation_pp,n_digits_pp,n_letters_pp,n_words_diff,n_sentences_diff,n_punctuation_diff,n_digits_diff,n_letters_diff,removals_idx,removals,insertions_idx,insertions,unchanged_idx,unchanged,n_segments_inserted,n_segments_removed,n_tokens_inserted,n_tokens_removed,is_truncation,any_phrase_capitalised,any_phrase_decapitalised,edit_distance_token_level
221,0,222,I like this movie,scro scro scro scro scro scro scro scro scro scro scro scro scro scro scro scro scro scro,0.706167,0.24464,0.755356,1,0,1,0.46152,-0.05277,0,-1.85767,0,0,221,1,2.33883,0.292354,0.25,8,4,20,4,0.28061,0.0441955,0.0336753,0.00583944,0.0273481,0.0103953,0.00199141,0.00595438,4.87491e-05,0.244649,0.00910938,0.033867,0.0048865,29,121,14,4,1,0,0,14,18,1,0,0,72,-14,0,0,0,-58,"[0, 1, 2, 3]",[I like this movie],"[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]",[scro scro scro scro scro scro scro scro scro scro scro scro scro scro scro scro scro scro],[],[],1,1,18,4,False,False,False,18



###############

random 1 

Original: I hate this apple
Original label 0
Unique paraphrases: 31
How the paraphrases change:


...

,idx,epoch,orig_l,pp_l,orig_truelabel_probs,pp_truelabel_probs,pp_predclass_probs,orig_label,pp_predclass,label_flip,vm_score,sts_score,reward,pp_logp,loss,batch_num,global_step,acc_num,loss_sum,loss_batch,label_flip_fraction,orig_length,orig_batch_size,pp_length,pp_batch_size,time_generate_pp,time_loss_fn,time_reward_fn,time_vm_scores,time_sts_scores,time_pp_logp,time_log_entropy,time_log_token_probabilities,time_loss_fn_loss_calc,time_backwards,time_calc_gradient_norm,time_opt_step_and_calc_param_norm,time_opt_step,idx_n_unique_pp,idx_n_pp_changes,epoch_of_first_label_flip,n_words_orig,n_sentences_orig,n_punctuation_orig,n_digits_orig,n_letters_orig,n_words_pp,n_sentences_pp,n_punctuation_pp,n_digits_pp,n_letters_pp,n_words_diff,n_sentences_diff,n_punctuation_diff,n_digits_diff,n_letters_diff,removals_idx,removals,insertions_idx,insertions,unchanged_idx,unchanged,n_segments_inserted,n_segments_removed,n_tokens_inserted,n_tokens_removed,is_truncation,any_phrase_capitalised,any_phrase_decapitalised,edit_distance_token_level
221,0,222,I like this movie,scro scro scro scro scro scro scro scro scro scro scro scro scro scro scro scro scro scro,0.706167,0.24464,0.755356,1,0,1,0.46152,-0.05277,0,-1.85767,0,0,221,1,2.33883,0.292354,0.25,8,4,20,4,0.28061,0.0441955,0.0336753,0.00583944,0.0273481,0.0103953,0.00199141,0.00595438,4.87491e-05,0.244649,0.00910938,0.033867,0.0048865,29,121,14,4,1,0,0,14,18,1,0,0,72,-14,0,0,0,-58,"[0, 1, 2, 3]",[I like this movie],"[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]",[scro scro scro scro scro scro scro scro scro scro scro scro scro scro scro scro scro scro],[],[],1,1,18,4,False,False,False,18



###############

low_sts 1 

Original: I do not like this movie
Original label 0
Unique paraphrases: 23
How the paraphrases change:


,,,,,,epoch
pp_l,pp_truelabel_probs,vm_score,sts_score,reward,label_flip,
This movie is not something I like.,0.91950,-0.02685,0.90957,0.47558,0,"[1, 4, 44, 126, 163]"
I don't like this movie.,0.92424,-0.03159,0.92286,0.47085,0,"[2, 3, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 33, 34, 35, 36, 38, 40, 41, 42, 43, 45, 46, 47, 48, 50, 51, 52, 54, 55, 57, 58, 59, 61, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 101, 102, 104, 105, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, ...]"
I do not not.,0.73804,0.15461,0.34834,0.00000,0,[11]
".)) of the, except, the';;;;;;;;",0.62290,0.26975,0.02015,0.00000,0,[21]
I don't like this movie,0.89997,-0.00732,0.95817,0.49298,0,"[32, 53, 56, 230, 249]"
?............,0.64133,0.25132,0.14184,0.00000,0,[37]
I don't like movies.,0.91475,-0.02210,0.67160,0.00000,0,[39]
""" "" ""."" """,0.65192,0.24073,0.07541,0.00000,0,[49]
or or by by,0.64258,0.25007,0.06084,0.00000,0,[60]


Best Paraphrase


,idx,epoch,orig_l,pp_l,orig_truelabel_probs,pp_truelabel_probs,pp_predclass_probs,orig_label,pp_predclass,label_flip,vm_score,sts_score,reward,pp_logp,loss,batch_num,global_step,acc_num,loss_sum,loss_batch,label_flip_fraction,orig_length,orig_batch_size,pp_length,pp_batch_size,time_generate_pp,time_loss_fn,time_reward_fn,time_vm_scores,time_sts_scores,time_pp_logp,time_log_entropy,time_log_token_probabilities,time_loss_fn_loss_calc,time_backwards,time_calc_gradient_norm,time_opt_step_and_calc_param_norm,time_opt_step,idx_n_unique_pp,idx_n_pp_changes,epoch_of_first_label_flip,n_words_orig,n_sentences_orig,n_punctuation_orig,n_digits_orig,n_letters_orig,n_words_pp,n_sentences_pp,n_punctuation_pp,n_digits_pp,n_letters_pp,n_words_diff,n_sentences_diff,n_punctuation_diff,n_digits_diff,n_letters_diff,removals_idx,removals,insertions_idx,insertions,unchanged_idx,unchanged,n_segments_inserted,n_segments_removed,n_tokens_inserted,n_tokens_removed,is_truncation,any_phrase_capitalised,any_phrase_decapitalised,edit_distance_token_level
402,1,103,I do not like this movie,Sac Sac Sac Sac Sac Sac Sac,0.892648,0.50375,0.503753,0,0,0,0.3889,-0.02998,0,-6.22549,0,0,102,0,1.56995,0.196244,0,8,4,10,4,0.135817,0.040889,0.03294,0.0055813,0.0269112,0.00783224,0.00130314,0.00412977,4.2852e-05,0.118678,0.0123139,2.13385e-05,1.43051e-06,23,60,1,6,1,0,0,19,7,1,0,0,21,-1,0,0,0,-2,"[0, 1, 2, 3, 4, 5]",[I do not like this movie],"[6, 7, 8, 9, 10, 11, 12]",[Sac Sac Sac Sac Sac Sac Sac],[],[],1,1,7,6,False,False,False,7


## Looking at common removals and insertions 

In [ ]:
def get_common_removals_and_insertions(df_concat): 
    idx = df_concat[['data_split','orig_l', 'pp_l']].drop_duplicates().index
    df_unique_pp = df_concat[['data_split','orig_l', 'pp_l','insertions', 'removals']].iloc[idx]
    def flatten_list(l): return [item for sublist in l for item in sublist] 
    removals_flat   =  flatten_list(df_unique_pp['removals'].values)
    insertions_flat =  flatten_list(df_unique_pp['insertions'].values)
    return pd.value_counts(removals_flat), pd.value_counts(insertions_flat)

In [ ]:
df_concat = _prepare_df_concat(df_d)
removals, insertions = get_common_removals_and_insertions(df_concat)

print("\n#### REMOVALS ####\n")
print(removals.head(10))
print("\n#### INSERTIONS ####\n")
print(insertions.head(10))


#### REMOVALS ####

I do not like this movie    15
I hate this apple           12
I love this                 12
I like this movie           11
not                         10
this                         9
this apple                   8
I love this apple            8
hate this                    6
this movie                   6
dtype: int64

#### INSERTIONS ####

.                   50
This                19
do n't like         10
n't                 10
apples               7
the                  5
....                 4
like                 3
movies               3
This is an apple     2
dtype: int64


Here you can look at a specific phrase and examples of where it appears. 

In [ ]:
def investigate_phrase(phrase, cname, n ): 
    mask = [phrase in strs for strs in df_concat[cname]]
    display_all(df_concat[mask].sample(n))

In [ ]:
investigate_phrase('not', 'removals', 4)

,idx,epoch,orig_l,pp_l,orig_truelabel_probs,pp_truelabel_probs,pp_predclass_probs,orig_label,pp_predclass,label_flip,vm_score,sts_score,reward,pp_logp,loss,batch_num,global_step,acc_num,loss_sum,loss_batch,label_flip_fraction,orig_length,orig_batch_size,pp_length,pp_batch_size,time_generate_pp,time_loss_fn,time_reward_fn,time_vm_scores,time_sts_scores,time_pp_logp,time_log_entropy,time_log_token_probabilities,time_loss_fn_loss_calc,time_backwards,time_calc_gradient_norm,time_opt_step_and_calc_param_norm,time_opt_step,idx_n_unique_pp,idx_n_pp_changes,epoch_of_first_label_flip,n_words_orig,n_sentences_orig,n_punctuation_orig,n_digits_orig,n_letters_orig,n_words_pp,n_sentences_pp,n_punctuation_pp,n_digits_pp,n_letters_pp,n_words_diff,n_sentences_diff,n_punctuation_diff,n_digits_diff,n_letters_diff,removals_idx,removals,insertions_idx,insertions,unchanged_idx,unchanged,n_segments_inserted,n_segments_removed,n_tokens_inserted,n_tokens_removed,is_truncation,any_phrase_capitalised,any_phrase_decapitalised,edit_distance_token_level,data_split
2826,1,127,I do not like this banana,I don't like this banana.,0.879454,0.92327,0.923266,0,0,0,-0.04381,0.94025,0.45881,-2.590769,1.188661,0,127,1,2.958575,0.369822,0.0,8,4,10,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,2,1.0,6,1,0,0,20,6,1,2,0,19,0,0,-2,0,1,[2],[not],"[3, 7]","[n't, .]","[0, 1, 4, 5, 6]","[I do, like this banana]",2,1,2,1,False,False,False,2,eval_valid
2907,1,208,I do not like this banana,I don't like this banana.,0.879454,0.92327,0.923266,0,0,0,-0.04381,0.94025,0.45881,-1.648664,0.756417,0,208,0,2.826527,0.353316,0.0,8,4,10,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,2,1.0,6,1,0,0,20,6,1,2,0,19,0,0,-2,0,1,[2],[not],"[3, 7]","[n't, .]","[0, 1, 4, 5, 6]","[I do, like this banana]",2,1,2,1,False,False,False,2,eval_valid
397,1,98,I do not like this movie,I don't like this movie.,0.892648,0.92424,0.924236,0,0,0,-0.03159,0.92286,0.47085,-2.927742,1.378524,0,97,1,4.913680,0.614210,0.0,8,4,10,4,0.135382,0.039967,0.032799,0.005523,0.026763,0.007054,0.001254,0.004113,0.000043,0.117263,0.010784,0.03557,0.005305,23,60,1.0,6,1,0,0,19,6,1,2,0,18,0,0,-2,0,1,[2],[not],"[3, 7]","[n't, .]","[0, 1, 4, 5, 6]","[I do, like this movie]",2,1,2,1,False,False,False,2,training_step
2770,1,71,I do not like this banana,I don't like banana.,0.879454,0.91965,0.919648,0,0,0,-0.04019,0.82136,0.46699,-2.963735,1.384024,0,71,1,5.369824,0.671228,0.0,8,4,10,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,2,1.0,6,1,0,0,20,5,1,2,0,15,1,0,-2,0,5,"[2, 5]","[not, this]","[3, 7]","[n't, .]","[0, 1, 4, 6]","[I do, like, banana]",2,2,2,2,False,False,False,3,eval_valid
